**Feature Extraction by using Mel-spectrogram**

In [ ]:
# !df -h


Filesystem      Size  Used Avail Use% Mounted on
overlay         226G   18G  209G   8% /
tmpfs            64M     0   64M   0% /dev
shm             165G     0  165G   0% /dev/shm
/dev/root       2.0G  1.2G  820M  59% /usr/sbin/docker-init
/dev/sda1       233G   31G  202G  14% /kaggle/input
tmpfs           168G  164K  168G   1% /var/colab
tmpfs           168G     0  168G   0% /proc/acpi
tmpfs           168G     0  168G   0% /proc/scsi
tmpfs           168G     0  168G   0% /sys/firmware
drive           226G   28G  198G  13% /content/drive


In [ ]:
# !rm -rf /content/tmp/*


In [ ]:
# !pip install librosa
# !pip install noisereduce
# !pip install audiomentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 920.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 30.0 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1


In [ ]:
# !pip install -q scikit-learn tensorflow matplotlib

In [ ]:





import pandas as pd
import os
import re
import concurrent.futures
from tqdm import tqdm
import librosa
import numpy as np
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import gc
import tempfile
import pickle
import pickle
from google.colab import files




In [ ]:

# Load the transcribed CSV file
file_path = '/content/drive/MyDrive/audio_files/Human_Grading_new_Data1.csv'
data = pd.read_csv(file_path)
print(len(data))

# Display the first few rows of the data
data.head()


92


,Participant,CHORD,ACHE,DEPOT,AISLE,BOUGUET,PSALM,CAPON,DENY,NAUSEA,...,GAUCHE,TOPIARY,LEVIATHAN,BEATIFY,PRELATE,SIDEREAL,DEMESNE,SYNCOPE,LABILE,CAMPANILE
0,1,0,0,0,0,0,0,1,0,0,...,0,0,1,1,1,0,1,1,1,1
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,0,1
2,3,0,0,0,0,0,1,0,0,0,...,1,0,1,1,1,1,1,1,1,1
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,1,1,0,1
4,5,0,0,0,0,0,0,1,0,0,...,0,0,1,1,1,1,1,1,1,1


In [ ]:
# Load the transcribed CSV files
file_path1 = '/content/drive/MyDrive/audio_files/Human_Grading_new_Data1.csv'
data1 = pd.read_csv(file_path1)

file_path2 = '/content/drive/MyDrive/audio_files/Human NART Grading New_Data2.csv'
data2 = pd.read_csv(file_path2)

# Directories containing the audio files
audio_dir1 = "/content/drive/MyDrive/audio_files/Dataset 1 (Cleaned with Separated Words)/NART Words Ss 1 to 71"
audio_dir2 = "/content/drive/MyDrive/audio_files/Dataset_2_Cleaned"




In [ ]:
def normalize_word(word: str) -> str:
    normalized = re.sub(r'^\d+', '', word.strip().replace(" ", "").replace("-", "").replace("_", "").lower())
    print(f"Normalizing word '{word}' to '{normalized}'")
    return normalized




In [ ]:
# build file mapping
def build_file_mapping(audio_dir: str) -> dict:
    file_mapping = {}
    for root, dirs, files in os.walk(audio_dir):
        for file in files:
            if file.endswith(".wav"):
                word = os.path.basename(root)
                normalized_word = normalize_word(word)
                file_path = os.path.join(root, file)
                if normalized_word not in file_mapping:
                    file_mapping[normalized_word] = []
                file_mapping[normalized_word].append(file_path)
    return file_mapping

# file mappings for both datasets
file_mapping1 = build_file_mapping(audio_dir1)
file_mapping2 = build_file_mapping(audio_dir2)

# Combine the mappings
file_mapping = {**file_mapping1, **file_mapping2}

Streaming output truncated to the last 5000 lines.
Normalizing word '21 HIATUS' to 'hiatus'
Normalizing word '21 HIATUS' to 'hiatus'
Normalizing word '21 HIATUS' to 'hiatus'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normalizing word '33 DRACHM' to 'drachm'
Normal

In [ ]:
# Define augmentation
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
    Shift(min_shift=-0.5, max_shift=0.5, p=0.5),
])

def augment_audio(y: np.ndarray, sr: int) -> np.ndarray:
    return augment(samples=y, sample_rate=sr)

# extract mel-spectrogram features from audio file
def extract_mel_spectrogram(file_path: str, n_mels: int = 128, fmax: int = 8000, chunk_duration: int = 5) -> tuple:
    y, sr = librosa.load(file_path, sr=None)
    if len(y) == 0:
        print(f"Warning: {file_path} contains no data")
    features = []
    num_chunks = (len(y) + chunk_duration * sr - 1) // (chunk_duration * sr)  # num_chunks is positive
    for start in range(0, len(y), chunk_duration * sr):
        chunk = y[start:start + chunk_duration * sr]
        if len(chunk) < chunk_duration * sr:
            chunk = np.pad(chunk, (0, chunk_duration * sr - len(chunk)))
        S = librosa.feature.melspectrogram(y=chunk, sr=sr, n_mels=n_mels, fmax=fmax)
        S_dB = librosa.power_to_db(S, ref=np.max)
        features.append(S_dB)
    # print(f"extract_mel_spectrogram: {len(features)} features, {num_chunks} chunks for file {file_path}")
    return features, num_chunks

# extract and augment features from audio file
def extract_and_augment_features(file_path: str, augmentations: int = 2, chunk_duration: int = 5) -> tuple:
    y, sr = librosa.load(file_path, sr=None)
    if len(y) == 0:
        print(f"Warning: {file_path} contains no data")
    features = []
    num_chunks = (len(y) + chunk_duration * sr - 1) // (chunk_duration * sr)
    for _ in range(augmentations):
        y_augmented = augment_audio(y, sr)
        for start in range(0, len(y_augmented), chunk_duration * sr):
            chunk = y_augmented[start:start + chunk_duration * sr]
            if len(chunk) < chunk_duration * sr:
                chunk = np.pad(chunk, (0, chunk_duration * sr - len(chunk)))
            S = librosa.feature.melspectrogram(y=chunk, sr=sr, n_mels=128, fmax=8000)
            S_dB = librosa.power_to_db(S, ref=np.max)
            features.append(S_dB)
    # print(f"extract_and_augment_features: {len(features)} features, {augmentations * num_chunks} chunks for file {file_path}")
    return features, augmentations * num_chunks

In [ ]:
def process_file(file_path: str, label: int, augmentations: int = 2, chunk_duration: int = 5) -> tuple:
    if label == 0:  # Only augment for corrected words
        features, num_chunks = extract_and_augment_features(file_path, augmentations, chunk_duration)
    else:
        features, num_chunks = extract_mel_spectrogram(file_path, chunk_duration=chunk_duration)
    labels = [label] * len(features)
    # print(f"process_file: {len(features)} features, {len(labels)} labels for file {file_path}")
    return features, labels

def process_data(data: pd.DataFrame, file_mapping: dict, max_workers: int = 4, batch_size: int = 50) -> tuple:
    all_features = []
    all_labels = []
    tasks = []

    # Collect tasks
    for _, row in data.iterrows():
        for word in data.columns[1:]:  # Skip the participant column
            label = row[word]
            normalized_word = normalize_word(word)
            if normalized_word in file_mapping:
                file_paths = file_mapping[normalized_word]  # Get the list of file paths for this word
                for file_path in file_paths:
                    if os.path.exists(file_path):
                        tasks.append((file_path, label))
            else:
                print(f"No files found for word {word} (normalized as {normalized_word})")

    with tempfile.TemporaryDirectory() as tempdir:
        temp_files = []

        # Process tasks in batches to save memory
        for i in range(0, len(tasks), batch_size):
            batch_tasks = tasks[i:i+batch_size]
            batch_features, batch_labels = process_batch(batch_tasks, max_workers)

            # Verify the batch features and labels before saving
            if len(batch_features) != len(batch_labels):
                print(f"Error: Mismatch in batch features and labels at batch {i}")
                continue

            # Save results to disk
            temp_file = os.path.join(tempdir, f'batch_{i}.pkl')
            with open(temp_file, 'wb') as f:
                pickle.dump((batch_features, batch_labels), f)
            temp_files.append(temp_file)

            gc.collect()  # Free up memory

        # Load results from disk and combine
        for temp_file in temp_files:
            with open(temp_file, 'rb') as f:
                batch_features, batch_labels = pickle.load(f)
                all_features.extend(batch_features)
                all_labels.extend(batch_labels)

    return all_features, all_labels

In [ ]:
def process_batch(tasks: list, max_workers: int = 4) -> tuple:
    features = []
    labels = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_file = {executor.submit(process_file, file_path, label): (file_path, label) for file_path, label in tasks}
        for future in tqdm(concurrent.futures.as_completed(future_to_file), total=len(future_to_file)):
            file_path, label = future_to_file[future]
            try:
                file_features, file_labels = future.result()
                features.extend(file_features)
                labels.extend(file_labels)
                # print(f"process_batch: Added {len(file_features)} features, {len(file_labels)} labels for file {file_path}")
            except Exception as exc:
                print(f"{file_path} generated an exception: {exc}")

    return features, labels

# Process both datasets
features1, labels1 = process_data(data1, file_mapping)
features2, labels2 = process_data(data2, file_mapping)

# Combine features and labels from both datasets
features = features1 + features2
labels = labels1 + labels2

print(f"Processed {len(features)} features and {len(labels)} labels")

Normalizing word 'CHORD' to 'chord'
Normalizing word 'ACHE' to 'ache'
Normalizing word 'DEPOT' to 'depot'
Normalizing word 'AISLE' to 'aisle'
Normalizing word 'BOUGUET' to 'bouguet'
No files found for word BOUGUET (normalized as bouguet)
Normalizing word 'PSALM' to 'psalm'
Normalizing word 'CAPON' to 'capon'
Normalizing word 'DENY' to 'deny'
Normalizing word 'NAUSEA' to 'nausea'
Normalizing word 'DEBT' to 'debt'
Normalizing word 'COURTEOUS' to 'courteous'
Normalizing word 'RAFEFY' to 'rafefy'
No files found for word RAFEFY (normalized as rafefy)
Normalizing word 'EQUIVOCAL' to 'equivocal'
Normalizing word 'NAIVE' to 'naive'
Normalizing word 'CATACOMB' to 'catacomb'
Normalizing word 'GAOLED' to 'gaoled'
Normalizing word 'THYME' to 'thyme'
Normalizing word 'HEIR' to 'heir'
Normalizing word 'RADIX' to 'radix'
Normalizing word 'ASSIGNATE' to 'assignate'
Normalizing word 'HIATUS' to 'hiatus'
Normalizing word 'SUBTLE' to 'subtle'
Normalizing word 'PROCREATE' to 'procreate'
Normalizing word '

100%|██████████| 18/18 [00:01<00:00, 12.10it/s]


Normalizing word 'CHORD' to 'chord'
Normalizing word 'ACHE' to 'ache'
Normalizing word 'DEPOT' to 'depot'
Normalizing word 'AISLE' to 'aisle'
Normalizing word 'BOUQUET' to 'bouquet'
Normalizing word 'PSALM' to 'psalm'
Normalizing word 'CAPON' to 'capon'
Normalizing word 'DENY' to 'deny'
Normalizing word 'NAUSEA' to 'nausea'
Normalizing word 'DEBT' to 'debt'
Normalizing word 'COURTEOUS' to 'courteous'
Normalizing word 'RAREFY' to 'rarefy'
Normalizing word 'EQUIVOCAL' to 'equivocal'
Normalizing word 'NAIVE' to 'naive'
Normalizing word 'CATACOMB' to 'catacomb'
Normalizing word 'GAOLED' to 'gaoled'
Normalizing word 'THYME' to 'thyme'
Normalizing word 'HEIR' to 'heir'
Normalizing word 'RADIX' to 'radix'
Normalizing word 'ASSIGNATE' to 'assignate'
Normalizing word 'HIATUS' to 'hiatus'
Normalizing word 'SUBTLE' to 'subtle'
Normalizing word 'PROCREATE' to 'procreate'
Normalizing word 'GIST' to 'gist'
Normalizing word 'GOUGE' to 'gouge'
Normalizing word 'SUPERFLUOUS' to 'superfluous'
Normalizin

100%|██████████| 5/5 [00:00<00:00, 17.61it/s]


Processed 368675 features and 368675 labels


In [ ]:
# Save the processed data to disk
# output_file = 'processed_data.pkl'
# with open(output_file, 'wb') as f:
#     pickle.dump((features, labels), f)

# # Download the file
# from google.colab import files
# files.download(output_file)

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define the destination path in Google Drive
destination_path = '/content/drive/MyDrive/audio_files/processed_data.pkl'

# Save the processed data directly to Google Drive
with open(destination_path, 'wb') as f:
    pickle.dump((features, labels), f)

print("File has been saved directly to the Google Drive at:", destination_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File has been saved directly to Google Drive at: /content/drive/MyDrive/audio_files/processed_data.pkl
